In [1]:
from __future__ import annotations

import time
import datetime
import os
import json
from copy import deepcopy

import numpy as np
import cst_python as cst
import toolpy as tp
from toolpy.integrations import groq

import gbyg as gg

Define LLM to use:

In [2]:
groq_interface = groq.GroqInterface(model=groq.GroqModel.LLAMA3_70B, n_retry=10)

registry = tp.llm.LLMRegistry()
registry.registry(model_name="llama3-70b", interface=groq_interface, default=True)

Create agent:

In [3]:
importance_threshould = 1

In [4]:
mind = cst.Mind()

gg.agent.agent_constructor(mind, importance_threshould)

for codelet in mind.code_rack.all_codelets:
    codelet.time_step = 100

memories_input = mind.raw_memory.get_all_of_type("MemoriesInput")[0]
memory_stream = mind.raw_memory.get_all_of_type("MemoryStream")[0]
agent_time_memory = mind.raw_memory.get_all_of_type("AgentTime")[0]
agent_info_memory = mind.raw_memory.get_all_of_type("AgentInfo")[0]
agent_summary_description = mind.raw_memory.get_all_of_type("AgentSummaryDescription")[0]
actual_place_memory = mind.raw_memory.get_all_of_type("ActualPlace")[0]
known_world = mind.raw_memory.get_all_of_type("KnownWorld")[0]
action_memory = mind.raw_memory.get_all_of_type("Action")[0]
current_observation_memory = mind.raw_memory.get_all_of_type("CurrentObservation")[0]
current_state_memory = mind.raw_memory.get_all_of_type("CurrentState")[0]

previous_day_summary = mind.raw_memory.get_all_of_type("PreviousDaySummary")[0]
plan = mind.raw_memory.get_all_of_type("Plan")[0]

Set initial agent info:

In [5]:
agent_info_memory.set_info({"name":"Alex", "age":25, "traits":"friendly, outgoing, hospitable"})
agent_time_memory.set_info(datetime.datetime.fromisoformat("2024-06-26T00:00:00").timestamp())
actual_place_memory.set_info("Bedroom")

-1

In [6]:
mind.start()

In [7]:
with open("data\\seed_memories.json") as file:
    seed_memories = json.load(file)

input_memories = []
for memory_name in seed_memories:
    memory = {"type": "reflection",
            "created": "2024-01-01 08:00:00",
            "description": seed_memories[memory_name]}

    input_memories.append(memory)

with open("data\\yesterday_routine.json") as file:
  yesterday_memories = json.load(file) 

#input_memories += deepcopy(yesterday_memories)

for m in input_memories:
    m["created"] = datetime.datetime.fromisoformat(m["created"]).timestamp()
    m["last_acessed"] = m["created"]

In [8]:
memories_input_queue = memories_input.get_info()
for memory in input_memories:
    memories_input_queue.append(memory)

while len(memory_stream.get_info()) != len(input_memories):
    time.sleep(0.1)

D:\Github\GDataByGAgents\src\gbyg\agent\retrieval\recency_scorer_tool.py:32: RuntimeWarning: invalid value encountered in divide
  scores = (scores-min_score)/(max_score-min_score)
c:\Python38\lib\site-packages\sentence_transformers\util.py:41: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:278.)
  b = torch.tensor(b)


Check stored memories:

In [9]:
for memory in memory_stream.get_info():
    because_of = ""
    if 'because_of' in memory: 
        because_of = memory['because_of']

    memory_str = f"{memory['index']} ({memory['type']}): {memory['description']} | because of: {because_of}"
    print(memory_str)

1 (reflection): Alex is a 25-year-old photographer known for his friendly, outgoing, and hospitable nature | because of: 
2 (reflection): Alex is passionated about capturing moments through the lens has made him a well-respected figure in the local photography scene | because of: 
3 (reflection): Alex enjoys experimenting with different styles and techniques, constantly pushing the boundaries of his creativity. | because of: 
4 (reflection): Alex has a particular fondness for photographing nature, finding inspiration in the beauty of landscapes, wildlife, and natural light. | because of: 
5 (reflection): Alex is a social butterfly who thrives on meeting new people and making lasting connections. | because of: 
6 (reflection): Alex has a knack for making others feel welcome and comfortable, whether he's hosting a casual get-together or a larger event. | because of: 
7 (reflection): Alex is warmth and enthusiasm for life are evident in everything he does, making him a beloved presence in

Generate previous day and agent summary:

In [10]:
summary_function = gg.agent.summary.SummaryFunction()

query = {"query_statement":"Alex's previous day",
         "statements":json.dumps(yesterday_memories)}

summary_result, _ = summary_function(query)

summary = f'''Yesteday, 26 June {datetime.datetime.fromisoformat(yesterday_memories[0]["created"]).strftime("%d %B")},
{summary_result['description']}.
Alex took several photos, preparing to later edit them and post them on social media.
Alex now plans to stay home the entire day.'''

In [11]:
last_summary_time = agent_summary_description.get_timestamp()

agent_time_memory.set_info(agent_time_memory.get_info()+gg.time_utils.TimeInterval.ONE_DAY)

previous_day_summary_content = {"summary":summary,
                                "generated_time":agent_time_memory.get_info()}

previous_day_summary.set_info(previous_day_summary_content)

while agent_summary_description.get_timestamp() == last_summary_time:
    time.sleep(0.1)

print(agent_summary_description.get_info()["summary"])

Name: Alex (age: 25)
Innate traits: friendly, outgoing, hospitable
Alex is a young, sociable, and welcoming photographer.
A 25-year-old photographer with a friendly and outgoing personality
There is no information provided about Alex's feelings about their recent progress in life based on the given statement.


Wait for plan and check plan:

In [12]:
while plan.get_info() == "":
    time.sleep(0.1) 

In [13]:
#Remove fisrt hour actions to avoid infinity loop bellow
while plan.get_info()[0]["hour"] == 0:
    del plan.get_info()[0]

In [14]:
for action in plan.get_info():
    hour = action["hour"]
    minute = action["minute"]
    action_action = action["action"]

    print(f"{hour}:{minute}: {action_action}")

8:0: Wake up
8:15: Have breakfast
9:0: Gather all photos from yesterday's outing
9:10: Organize photos into folders
9:20: Start editing photos
12:0: Start taking a lunch break
12:30: Continue taking a lunch break
13:0: Resume editing photos
17:0: Take a short break to relax and unwind
18:0: Continue editing photos
20:0: Select and gather photos for social media
20:10: Organize photos into logical folders
20:20: Write captions for each photo
20:30: Review and edit captions for clarity and grammar
20:40: Upload photos and captions to social media
21:0: Review photo selection for posting
21:20: Finalize photo selection for posting
22:0: Prepare for bed
22:30: Wind down and relax before sleeping
22:50: Get into bed, feeling accomplished


Load environment:

In [15]:
with open("data\\environment.json") as file:
    alex_house = json.load(file)

alex = {"name":"Alex", "type":"agent", "description":"human", 'state':'sleeping in the bed.'}

alex_house["Alex's House"]["Bedroom"].append(alex)

agent_names = ["Alex"]

In [16]:
from collections import deque
from copy import deepcopy

known_world_tree = deepcopy(alex_house)

stack = deque()
for room_name in known_world_tree:
    room = known_world_tree[room_name]
    stack.append(room)

while len(stack) > 0:
    room = stack.pop()
    if isinstance(room, list):
        for object in room:
            del object["state"]
    else:
        for subroom_name in room:
            subroom = room[subroom_name]
            stack.append(subroom)

In [17]:
known_world.set_info(known_world_tree)

-1

Agent initial state:

In [18]:
observation_memory = {"index": 0,
    "type": "event",
    "created": agent_time_memory.get_info(),
    "description": f"Alex is {alex['state']}"}

input_memories.append(observation_memory)

In [19]:
action_memory.get_info()

''

Get simulation information:

In [20]:
first_action = plan.get_info()[0]
last_action = plan.get_info()[-1]

agent_time = agent_time_memory.get_info()

now = datetime.datetime.fromtimestamp(agent_time)

now = now.replace(hour=first_action["hour"], minute=first_action["minute"])
agent_time_memory.set_info(now.timestamp())

simulation_end_time = datetime.datetime.fromtimestamp(agent_time)
simulation_end_time = simulation_end_time.replace(hour=last_action["hour"], minute=last_action["minute"]).timestamp()

Wait for agent generate first action:

In [21]:
while action_memory.get_info() == "":
    time.sleep(0.1)

last_action_time = action_memory.get_timestamp()

Create environment:

In [22]:
env = gg.environment.Environment(alex_house, agent_names)

Log function:

In [23]:
def encoder(obj):
    if isinstance(obj, np.integer):
            return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return []
    
    return obj.__dict__

sensor_result = {}

def dump_log(path:str):
    log = {}
    log["agent_time"] = agent_time_memory.get_info()
    log["action"] = action_memory.get_info()
    log["agent_summary_description"] = agent_summary_description.get_info()
    log["state"] = current_state_memory.get_info()
    log["curret_observation"] = current_observation_memory.get_info()
    log["current_place"] = actual_place_memory.get_info()
    log["sensor_result"] = sensor_result
    log["environment_tree"] = env._environment_tree
    

    log["memory_stream"] = list(memory_stream.get_info())

    if not os.path.isdir(path):
        os.mkdir(path)

    log_path = os.path.join(path, str(log["agent_time"])+".json")
    file = open(log_path, "w")
    json.dump(log, file, default=encoder)
    file.close()


In [24]:
dump_log("logs")

Create sensor:

In [25]:
sound_sensor = gg.sensors.SoundSensor()

Simulate:

In [26]:
while agent_time_memory.get_info() < simulation_end_time:
    print("Current time:", agent_time_memory.get_info())

    #Step the env
    agent_actions = {"Alex":action_memory.get_info()}

    print("env step|", end="")
    obsevation = env.step(agent_actions)
    alex_observation = obsevation["Alex"]

    #Insert memories
    last_memory_stream_size = len(memory_stream.get_info())

    observation_memory = {"type": "event",
        "created": agent_time_memory.get_info(),
        "description": f"Alex is {alex_observation['state']}"}

    memories_input_queue.append(observation_memory)

    observation_memory = {"type": "event",
        "created": agent_time_memory.get_info(),
        "description": alex_observation['observation'],
        "last_acessed":agent_time_memory.get_info()}

    memories_input_queue.append(observation_memory)

    current_state_memory.set_info(alex_observation["state"])
    current_observation_memory.set_info(alex_observation['observation'])
    actual_place_memory.set_info(alex_observation["place"])

    #Update agent time
    current_time = agent_time_memory.get_info()

    if alex_observation["duration"] == 0:
        alex_observation["duration"] = 1

    current_time += alex_observation["duration"]
    agent_time_memory.set_info(current_time)

    #Get sensor output
    query = {"action":agent_actions["Alex"],
            "state":alex_observation['state'],
            "observation":alex_observation['observation'],
            "last_acessed":agent_time_memory.get_info()}

    print("sensor step|", end="")
    sensor_result, _ = sound_sensor(query)

    #Wait for agent processing
    print("waiting for agent")
    while action_memory.get_timestamp() == last_action_time or len(memory_stream.get_info()) < last_memory_stream_size+2:
        time.sleep(0.1)

    last_action_time = action_memory.get_timestamp()

    dump_log("logs")

Current time: 1719486000.0
env step|

D:\Github\GDataByGAgents\src\gbyg\environment\tools\object_mover.py:57: UserWarning: Place None not found. Aborting change.
  warnings.warn(f"Place {place} not found. Aborting change.")


sensor step|waiting for agent
Current time: 1719486002.0
env step|sensor step|waiting for agent
Current time: 1719486004.0
env step|sensor step|waiting for agent
Current time: 1719486006.0
env step|sensor step|waiting for agent
Current time: 1719486008.5
env step|Traceback (most recent call last):
  File "D:\Github\CST-Python\src\cst_python\core\entities\codelet.py", line 450, in _scheduled_run
    self.proc()
  File "D:\Github\GDataByGAgents\src\gbyg\agent\retrieval\retrieval_codelet.py", line 83, in proc
    timestamps.append(memory["last_acessed"])
KeyError: 'last_acessed'

sensor step|waiting for agent
Current time: 1719486013.5
env step|

D:\Github\GDataByGAgents\src\gbyg\environment\tools\object_mover.py:67: UserWarning: Origin object Alex not found in Bedroom. Aborting change.
  warnings.warn(f"Origin object {object_name} not found in {original_place_name}. Aborting change.")


sensor step|waiting for agent
Current time: 1719486023.5
env step|

AttributeError: 'NoneType' object has no attribute 'keys'

In [27]:
mind.shutdown()